In [1]:
import glob
import psycopg2
import json

In [2]:
thesis_path = r"C:\Users\cvaka\OneDrive\Master\Thesis"
#thesis_path = r"C:\Users\Christophe\OneDrive\Master\Thesis"
table = "polygons"

In [3]:
conn = psycopg2.connect( \
    database="thesis",
    user = "postgres", 
    password = "jonp8UMs8qDV4jEcwOC0",
    host = "localhost"
    )
cur = conn.cursor()

In [5]:
query = """
DROP TABLE IF EXISTS {table};

CREATE TABLE {table} 
(
    id bigserial,
    name text,
    polygon polygon
);
""".format(table=table)
cur.execute(query)
conn.commit()

In [6]:
paths = glob.glob(thesis_path + r"\Data\\" + "polygons\\" + "*.json")
polygons = {}
for i, path in enumerate(paths): # Find all ast.gz files
    with open(path, "r") as f: # Open decode and split into lines
        polygons.update(json.load(f))

In [7]:
# Ugly solution (legacy) to deal with one name for multiple polygons
polygons_tuple = []
for polygon in polygons:
    if type(polygons[polygon][0][0])==list:
        for j in polygons[polygon]:
            polygons_tuple.append((polygon, j))
    else:
        polygons_tuple.append((polygon, polygons[polygon]))

In [8]:
query = "INSERT INTO {} (name, polygon) VALUES {}".format(
    table, 
    str(polygons_tuple)[1:-1].replace("[","(").replace("]", ")").replace("((", "'((").replace("))", "))'"))
cur.execute(query)
conn.commit()